In [3]:
import pytraj as pt 

top = "/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C263CsDJQucZ_job_004_END.pdb"; 
traj = "/home/yzhang/zhang/MyTrajs/BFL-1/batch3/C263CsDJQucZ_job_004_traj.nc";
traj = pt.load(traj, top=top, mask=":1-151,:LIG", stride=150)
traj

pytraj.Trajectory, 7 frames: 
Size: 0.000388 (GB)
<Topology: 2479 atoms, 152 residues, 2 mols, PBC with box type = cubic>
           

In [128]:
import numpy as np 

def generate_3d_grid(grid_size, spacing):
  x_vals = np.arange(0, grid_size[0]+1e-8, spacing); 
  y_vals = np.arange(0, grid_size[1]+1e-8, spacing); 
  z_vals = np.arange(0, grid_size[2]+1e-8, spacing); 
  points = []
  for x in x_vals:
    for y in y_vals:
      for z in z_vals:
        points.append([x, y, z])
  return np.array(points)
  
def select_by_dist(coord, limit):
  return [np.prod(i) for i in coord < limit]

def genslices(traj, center, cube_length): 
  center = np.array(center)
  upperbound = np.array(center) + cube_length/2; 
  lowerbound = np.array(center) - cube_length/2; 
  ret_arr = []
  idx_arr = []
  for xyz in traj.xyz: 
    ubstate = np.array([np.prod(i) for i in xyz < upperbound])
    lbstate = np.array([np.prod(i) for i in xyz > lowerbound])
    state = [bool(i) for i in ubstate*lbstate]
    ret_arr.append(xyz[state])
    idx_arr.append(state)
  return ret_arr, idx_arr

def group_residues(input_lst, threshold = 1):
  count = 0
  retlst = {}
  for idx, val in enumerate(input_lst): 
    if idx < len(input_lst)-1 and (input_lst[idx+1] - val <= threshold): 
      retlst[val] = count; 
    else: 
      retlst[val] = count; 
      count += 1; 
  return retlst
def most_abundant(lst): 
  uniq, counts = np.unique(lst, return_counts=True)
  return uniq[np.argmax(counts)]

def map_to_groups(traj, slices, idxs, points, threshold=1.5):
  # Iterate every frame; 
  resid_arr = np.array([i.resid for i in traj.top.atoms]); 
  retarr = []
  for coord, idx_arr in zip(slices,idxs): 
    # Obtain the atoms and group the atoms by residue; 
    residlst = resid_arr[idx_arr]; 
    residues, counts = np.unique(residlst, return_counts=True); 
    group_dict = group_residues(residues, threshold=3); 
    groupidlst = np.array([group_dict[i] for i in residlst]); 

    # Iterate points inside the aligned box; 
    group_arr = []
    for p in points: 
      atomwithincutoff = [bool(np.prod(i)) for i in np.abs(p - coord) < threshold]
      if np.sum(atomwithincutoff) > 0:
        groupnr = most_abundant(groupidlst[atomwithincutoff])
      else: 
        groupnr = -1
      group_arr.append(groupnr)
    groupconfig = np.unique(group_arr, return_counts=True)
    retarr.append(group_arr); 
  return np.array(retarr)


grid_length = 6; 
grid_size = [grid_length] * 3; 
grid_center = [30,32,14]; 
spacing = 0.3; 
points = generate_3d_grid(grid_size, spacing); 
aligned_points = (points - (grid_length/2)) + grid_center; 
slices, idxs = genslices(traj, grid_center, grid_length); 

grouped_arr = map_to_groups(traj, slices, idxs, aligned_points); 

print(theret[1])
print(points)
print(len(points))

[[32.81624222 31.32039261 15.94249916]
 [30.60195541 29.59212685 15.12362766]
 [31.43644142 30.26342964 15.68272495]
 ...
 [31.04022217 33.51049042 13.67857742]
 [29.76158333 33.99271393 16.41277695]
 [31.26522636 33.02267456 15.95958805]]
[[0.  0.  0. ]
 [0.  0.  0.3]
 [0.  0.  0.6]
 ...
 [6.  6.  5.4]
 [6.  6.  5.7]
 [6.  6.  6. ]]
9261


In [98]:
a = [9,10,15,16,18, 50, 115, 116, 151]

# The order of residue id is ordered. 

# print(a)
# def group_residues(input_lst, threshold = 1):
#   count = 0
#   retlst = []
#   for idx, val in enumerate(input_lst): 
#     if idx < len(input_lst)-1 and (input_lst[idx+1] - val <= threshold): 
#       retlst.append(count)
#     else: 
#       retlst.append(count)
#       count += 1; 
#   return retlst
retlst = group_residues(a,3); 
print(retlst)
  

  

most_abundant([2, 2, 2, 2, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 11, 1, 1, 1, 1, 1, 1, 1, ])


{9: 0, 10: 0, 15: 1, 16: 1, 18: 1, 50: 2, 115: 3, 116: 3, 151: 4}


2

In [174]:
import numpy as np
from scipy.optimize import least_squares

def calc_residuals(params, points):
    center = params[:3]
    radii = params[3:6]
    rotation = params[6:].reshape(3, 3)
    transformed_points = np.dot(points - center, rotation)
    residuals = np.sum(transformed_points**2 / radii**2, axis=1) - 1
    return residuals

def fit_ellipsoid_lsq(points):
    center_init = np.mean(points, axis=0)
    cov_matrix = np.cov(points, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    radii_init = np.sqrt(eigenvalues)
    rotation_init = eigenvectors
    params_init = np.concatenate((center_init, radii_init, rotation_init.ravel()))
    params_opt = least_squares(calc_residuals, params_init, args=(points,), method='lm')
    return params_opt.x
fit_ellipsoid_lsq(slices[1])

array([ 1.19093970e+02,  2.65340562e+01, -1.47808401e+02, ...,
        6.41482398e-03,  5.89649610e-02,  1.25387104e-02])

In [173]:
import numpy as np
from scipy import optimize

def calc_residuals(params, points):
    center = params[:3]
    radius = params[3]
    residuals = np.linalg.norm(points - center, axis=1) - radius
    return residuals

def fit_sphere_lsq(points):
    center_init = np.mean(points, axis=0)
    radius_init = np.mean(np.linalg.norm(points - center_init, axis=1))
    params_init = np.concatenate((center_init, [radius_init]))
    params_opt, _ = optimize.leastsq(calc_residuals, params_init, args=(points,))
    return params_opt
print(np.mean(slices[1], axis=0))
fit_sphere_lsq(slices[1])

[29.40605184 31.57590244 15.0050744 ]


array([29.96669023, 31.48440961, 14.21818274,  2.71645009])